# TrustyAI Evaluation Demo

This notebook demonstrates how to use the TrustyAI SDK's local evaluation functionality to evaluate language models using the LM Evaluation Harness.

## Prerequisites

Make sure you have installed TrustyAI with evaluation support:

```bash
pip install .[eval]
```

Or for all features:

```bash
pip install .[all]
```


## 1. Basic Setup and Imports

First, let's import the necessary modules and check what evaluation providers are available.


In [2]:
from pprint import pprint

from trustyai import Providers
from trustyai.core import DeploymentMode
from trustyai.core.eval import EvaluationProviderConfig

Explore the new Providers class:

In [3]:
print("=== TrustyAI Providers Class ===")
print("\nAvailable provider types:")
for provider_type in dir(Providers):
    print(f"  - {provider_type}")

print(f"\nAvailable evaluation providers:")
for provider_name in dir(Providers.eval):
    print(f"  - {provider_name}")

print(f"\nWe'll use: Providers.eval.LMEvalProvider")
print("The deployment mode in the config will determine whether it runs locally or on Kubernetes.")


=== TrustyAI Providers Class ===

Available provider types:
  - bias_detection
  - eval
  - evaluation
  - explainability

Available evaluation providers:
  - LMEvalProvider
  - Lm-evalProvider

We'll use: Providers.eval.LMEvalProvider
The deployment mode in the config will determine whether it runs locally or on Kubernetes.


## 3. Initialise the Local Evaluation Provider

Now let's create and initialise the local evaluation provider using the new organised Providers class.


In [4]:
# Create the evaluation provider using the new Providers class
# This automatically handles local vs Kubernetes deployment based on config
provider = Providers.eval.LMEvalProvider()

# Initialise the provider (this will check if lm-eval is available)
try:
    provider.initialize()
    print("✓ Evaluation provider initialised successfully!")
    print(f"Provider class: {provider.__class__.__name__}")
    print(f"Provider type: {provider.get_provider_type()}")
    print(f"Supported deployment modes: {[mode.value for mode in provider.get_supported_deployment_modes()]}")
except ImportError as e:
    print(f"✗ Error initialising provider: {e}")
    print("Please install evaluation dependencies: pip install .[eval]")


/home/rui/Sync/code/rh/trusty/trustyai-sdk/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Evaluation provider initialised successfully!
Provider class: LMEvalProvider
Provider type: eval
Supported deployment modes: ['local', 'kubernetes']


## 4. Explore Available Tasks and Metrics

Let's see what evaluation tasks and metrics are available through the provider.


In [5]:
# List available evaluation datasets/tasks
available_tasks = provider.list_available_datasets()
print(f"Number of available tasks: {len(available_tasks)}")
print("\nFirst 10 available tasks:")
for task in sorted(available_tasks)[:10]:
    print(f"  - {task}")

if len(available_tasks) > 10:
    print(f"  ... and {len(available_tasks) - 10} more tasks")


Number of available tasks: 0

First 10 available tasks:


In [6]:
# List available metrics
available_metrics = provider.list_available_metrics()
print(f"Available metrics ({len(available_metrics)}):")
for metric in available_metrics:
    print(f"  - {metric}")


Available metrics (13):
  - acc
  - acc_norm
  - perplexity
  - bleu
  - rouge
  - exact_match
  - f1
  - precision
  - recall
  - matthews_correlation
  - multiple_choice_grade
  - wer
  - ter


## 5. Basic Evaluation Example

Let's run a basic evaluation using a small model and a simple task. We'll use google/flan-t5-base (a small model) and the HellaSwag task for demonstration.


In [7]:
# Create evaluation configuration
config = EvaluationProviderConfig(
    evaluation_name="arc_easy",
    model="google/flan-t5-base",  # Small model for quick evaluation
    tasks=["arc_easy"],  # Common sense reasoning task
    limit=5,  # Limit to 5 examples for quick demonstration
    metrics=["acc", "acc_norm"],  # Accuracy metrics
    device="cpu",  # Use CPU to avoid GPU requirements - this should now be respected!
    deployment_mode=DeploymentMode.LOCAL,
    batch_size=1,  # Small batch size for stability
    num_fewshot=0  # Zero-shot evaluation
)

print("Configuration created:")
print(f"  Model: {config.model}")
print(f"  Tasks: {config.tasks}")
print(f"  Metrics: {config.metrics}")
print(f"  Device: {config.device} (should be 'cpu'!)")
print(f"  Limit: {config.limit} examples")
print(f"  Batch size: {config.get_param('batch_size')}")
print(f"  Few-shot examples: {config.get_param('num_fewshot')}")


Configuration created:
  Model: google/flan-t5-base
  Tasks: ['arc_easy']
  Metrics: ['acc', 'acc_norm']
  Device: cpu (should be 'cpu'!)
  Limit: 5 examples
  Batch size: 1
  Few-shot examples: 0


Run the evaluation

This may take a few minutes as the model needs to be downloaded and loaded.


In [8]:
try:
    results = provider.evaluate(config)
    print("\n✓ Evaluation completed successfully!")
except Exception as e:
    print(f"\n✗ Evaluation failed: {e}")
    results = None


2025-06-16:20:51:46,480 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-16:20:51:46,899 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-16:20:51:52,213 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-16:20:51:52,214 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-16:20:51:55,521 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of arc_easy from None to 0
2025-06-16:20:51:55,521 INFO     [lm_eval.api.task:420] Building contexts for arc_easy on rank 0...
100%|██████████| 5/5 [00:00<00:00, 2560.00it/s]
2025-06-16:20:51:55,525 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 20/20 [00:02<00:00,  9.74it/s]



✓ Evaluation completed successfully!


## 6. Deployment Mode Demonstration

The advantage of using `Providers.eval.LMEvalProvider` is that the same provider can handle both local and Kubernetes deployments. The deployment mode is specified in the configuration, not in the provider selection.

Let's demonstrate this by showing how you would configure for different deployment modes (we'll only run the local one for this demo).


In [9]:
# Same provider, different deployment modes!

# Configuration for LOCAL deployment
local_config = EvaluationProviderConfig(
    evaluation_name="local_demo",
    model="google/flan-t5-base",
    tasks=["arc_easy"],
    limit=3,
    metrics=["acc"],
    deployment_mode=DeploymentMode.LOCAL,  # This determines the implementation
    device="cpu",
    batch_size=1,
    num_fewshot=0
)

# Configuration for KUBERNETES deployment (same provider!)
kubernetes_config = EvaluationProviderConfig(
    evaluation_name="kubernetes_demo", 
    model="google/flan-t5-base",
    tasks=["arc_easy"],
    limit=3,
    metrics=["acc"],
    deployment_mode=DeploymentMode.KUBERNETES,  # Different deployment mode
    # Kubernetes-specific parameters would go here
    # namespace="trustyai",
    # cpu="1000m",
    # memory="2Gi"
)

print("📍 LOCAL Configuration:")
print(f"  Deployment mode: {local_config.deployment_mode.value}")
print(f"  Device: {local_config.device}")

print("\n🚀 KUBERNETES Configuration:")  
print(f"  Deployment mode: {kubernetes_config.deployment_mode.value}")
print(f"  Model: {kubernetes_config.model}")

print(f"\n✨ Same provider handles both: {provider.__class__.__name__}")
print("The provider automatically delegates based on deployment_mode!")


📍 LOCAL Configuration:
  Deployment mode: local
  Device: cpu

🚀 KUBERNETES Configuration:
  Deployment mode: kubernetes
  Model: google/flan-t5-base

✨ Same provider handles both: LMEvalProvider
The provider automatically delegates based on deployment_mode!


In [10]:
# Display results if evaluation was successful
if results:
    print("\nEvaluation Results:")
    print("==================")
    
    # Pretty print the results
    if 'results' in results:
        for task_name, task_results in results['results'].items():
            print(f"\nTask: {task_name}")
            print("-" * (len(task_name) + 6))
            for metric, value in task_results.items():
                if isinstance(value, (int, float)):
                    print(f"  {metric}: {value:.4f}")
                else:
                    print(f"  {metric}: {value}")
    else:
        print("Raw results:")
        pprint(results)



Evaluation Results:

Task: arc_easy
--------------
  alias: arc_easy
  acc,none: 0.6000
  acc_stderr,none: 0.2449
  acc_norm,none: 0.6000
  acc_norm_stderr,none: 0.2449


## 7. Multi-Task Evaluation

Let's run an evaluation on multiple tasks to see how the model performs across different capabilities.


In [11]:
# Configuration for multi-task evaluation
multi_task_config = EvaluationProviderConfig(
    evaluation_name="multi_task_demo",
    model="google/flan-t5-base",
    tasks=[
        "hellaswag",    # Common sense reasoning
        "arc_easy",     # Science questions (easy)
        "winogrande"    # Pronoun resolution
    ],
    limit=3,  # Very small limit for quick demo
    metrics=["acc", "acc_norm"],
    device="cpu",
    deployment_mode=DeploymentMode.LOCAL,
    batch_size=1,
    num_fewshot=0
)

print("Multi-task evaluation configuration:")
print(f"  Tasks: {multi_task_config.tasks}")
print(f"  Limit per task: {multi_task_config.limit} examples")


Multi-task evaluation configuration:
  Tasks: ['hellaswag', 'arc_easy', 'winogrande']
  Limit per task: 3 examples


In [12]:
# Run multi-task evaluation
print("Running multi-task evaluation...")

try:
    multi_results = provider.evaluate(multi_task_config)
    print("\n✓ Multi-task evaluation completed!")
    
    # Display results for each task
    if 'results' in multi_results:
        print("\nResults Summary:")
        print("================")
        
        for task_name, task_results in multi_results['results'].items():
            print(f"\n{task_name.upper()}:")
            for metric, value in task_results.items():
                if isinstance(value, (int, float)):
                    print(f"  {metric}: {value:.4f}")
                    
except Exception as e:
    print(f"\n✗ Multi-task evaluation failed: {e}")


2025-06-16:20:53:28,946 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


Running multi-task evaluation...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-16:20:53:29,219 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-16:20:53:34,535 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-16:20:53:34,536 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-16:20:53:43,573 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of winogrande from None to 0
2025-06-16:20:53:43,573 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of arc_easy from None to 0
2025-06-16:20:53:43,574 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of hellaswag from None to 0
2025-06-16:20:53:43,608 INFO     [lm_eval.api.task:420] Building contexts for winogrande on rank 0...
100%|██████████| 3/3 [00:00<00:00, 14043.43it/s]
2025-06-16:20:53:43,612 INFO     [lm_eval.api.task:420] Building co


✓ Multi-task evaluation completed!

Results Summary:

ARC_EASY:
  acc,none: 0.6667
  acc_stderr,none: 0.3333
  acc_norm,none: 0.6667
  acc_norm_stderr,none: 0.3333

HELLASWAG:
  acc,none: 0.3333
  acc_stderr,none: 0.3333
  acc_norm,none: 0.3333
  acc_norm_stderr,none: 0.3333

WINOGRANDE:
  acc,none: 0.6667
  acc_stderr,none: 0.3333


## 8. Few-Shot Evaluation

Let's demonstrate few-shot evaluation, where we provide examples to the model before asking it to perform the task.


In [13]:
# Configuration for few-shot evaluation
few_shot_config = EvaluationProviderConfig(
    evaluation_name="few_shot_demo",
    model="google/flan-t5-base",
    tasks=["hellaswag"],
    limit=3,
    metrics=["acc", "acc_norm"],
    device="cpu",
    deployment_mode=DeploymentMode.LOCAL,
    batch_size=1,
    num_fewshot=2  # Provide 2 examples before each test question
)

print("Few-shot evaluation configuration:")
print(f"  Task: {few_shot_config.tasks[0]}")
print(f"  Few-shot examples: {few_shot_config.get_param('num_fewshot')}")
print(f"  Test examples: {few_shot_config.limit}")


Few-shot evaluation configuration:
  Task: hellaswag
  Few-shot examples: 2
  Test examples: 3


In [14]:
# Run few-shot evaluation
print("Running few-shot evaluation...")

try:
    few_shot_results = provider.evaluate(few_shot_config)
    print("\n✓ Few-shot evaluation completed!")
    
    # Display results
    if 'results' in few_shot_results:
        task_name = list(few_shot_results['results'].keys())[0]
        task_results = few_shot_results['results'][task_name]
        
        print(f"\nFew-shot Results for {task_name}:")
        print("=" * (len(task_name) + 23))
        
        for metric, value in task_results.items():
            if isinstance(value, (int, float)):
                print(f"  {metric}: {value:.4f}")
                
except Exception as e:
    print(f"\n✗ Few-shot evaluation failed: {e}")


2025-06-16:20:53:59,666 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


Running few-shot evaluation...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-16:20:53:59,935 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-16:20:54:05,676 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-16:20:54:05,676 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-16:20:54:08,879 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of hellaswag from None to 2
2025-06-16:20:54:08,879 INFO     [lm_eval.api.task:420] Building contexts for hellaswag on rank 0...
100%|██████████| 3/3 [00:00<00:00, 1266.65it/s]
2025-06-16:20:54:08,885 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 12/12 [00:01<00:00,  9.78it/s]



✓ Few-shot evaluation completed!

Few-shot Results for hellaswag:
  acc,none: 0.3333
  acc_stderr,none: 0.3333
  acc_norm,none: 0.3333
  acc_norm_stderr,none: 0.3333


## 9. Comparing Models

Let's compare the performance of different models on the same task.


In [15]:
# List of models to compare (using small models for quick evaluation)
models_to_compare = [
    "google/flan-t5-base",
    "google/flan-t5-small"
]

comparison_results = {}

for model in models_to_compare:
    print(f"\nEvaluating {model}...")
    
    config = EvaluationProviderConfig(
        evaluation_name=f"comparison_{model}",
        model=model,
        tasks=["hellaswag"],
        limit=3,
        metrics=["acc", "acc_norm"],
        device="cpu",
        deployment_mode=DeploymentMode.LOCAL,
        batch_size=1,
        num_fewshot=0
    )
    
    try:
        results = provider.evaluate(config)
        if 'results' in results:
            task_results = results['results']['hellaswag']
            comparison_results[model] = task_results
            print(f"  ✓ {model} evaluation completed")
        else:
            print(f"  ✗ {model} evaluation returned unexpected format")
    except Exception as e:
        print(f"  ✗ {model} evaluation failed: {e}")
        comparison_results[model] = None


2025-06-16:20:54:16,668 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'



Evaluating google/flan-t5-base...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-16:20:54:16,954 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-16:20:54:22,399 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-16:20:54:22,400 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-16:20:54:25,544 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of hellaswag from None to 0
2025-06-16:20:54:25,544 INFO     [lm_eval.api.task:420] Building contexts for hellaswag on rank 0...
100%|██████████| 3/3 [00:00<00:00, 4800.81it/s]
2025-06-16:20:54:25,548 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 12/12 [00:00<00:00, 18.98it/s]
2025-06-16:20:54:26,663 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


  ✓ google/flan-t5-base evaluation completed

Evaluating google/flan-t5-small...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-16:20:54:27,045 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-16:20:54:33,320 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-16:20:54:33,320 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-16:20:54:36,278 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of hellaswag from None to 0
2025-06-16:20:54:36,279 INFO     [lm_eval.api.task:420] Building contexts for hellaswag on rank 0...
100%|██████████| 3/3 [00:00<00:00, 4463.61it/s]
2025-06-16:20:54:36,283 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 12/12 [00:04<00:00,  2.59it/s]


  ✓ google/flan-t5-small evaluation completed


In [16]:
# Display comparison results
print("\nModel Comparison Results:")
print("========================")
print(f"{'Model':<15} {'Accuracy':<10} {'Acc (Norm)':<10}")
print("-" * 35)

for model, results in comparison_results.items():
    if results:
        acc = results.get('acc', 'N/A')
        acc_norm = results.get('acc_norm', 'N/A')
        
        acc_str = f"{acc:.4f}" if isinstance(acc, (int, float)) else str(acc)
        acc_norm_str = f"{acc_norm:.4f}" if isinstance(acc_norm, (int, float)) else str(acc_norm)
        
        print(f"{model:<15} {acc_str:<10} {acc_norm_str:<10}")
    else:
        print(f"{model:<15} {'Failed':<10} {'Failed':<10}")



Model Comparison Results:
Model           Accuracy   Acc (Norm)
-----------------------------------
google/flan-t5-base N/A        N/A       
google/flan-t5-small N/A        N/A       
